In [ ]:
using Revise
include("HCAS_Reachability.jl");

## Setup
Generate memory-mapped files representing
1. Approximation of neural network policy (what advisories are given in each cell)
2. Cell dynamics (which cells are reachable from a given cell given an advisory)

In [ ]:
# There are two methods to approximate the neural network policy:

# This method evaluates a point within each cell to determine one action given within each cell
# This is not an over-approximation, but is faster than ReluVal and useful for debugging
writeNetworkActionsMmap(folder="NetworkApprox",nnetFolder="../networks",ver=6,hu=25,epochs=3000,useReluVal=false);

# This method reads in the output of running the runReluVal.sh script, which runs ReluVal to compute all advisories given
# within each cell.
# This is an over-approximation of the neural network policy and can be run after running the runReluVal.sh script.
writeNetworkActionsMmap(folder="NetworkApprox",reluvalFolder="../ReluVal/Results",ver=6,hu=25,epochs=3000,useReluVal=true);

In [ ]:
# Takes a little while to run, but only needs to be run once for a given delta value, assuming
# speed is constant
delta = 0.0
writeReachDynamicsMmap(folder="ReachDynamics",ras=ACTIONS,ver=1,delta=delta,v0=200.0,v1=180.0);

## Run Experiments

In [ ]:
# Key Parameters
delta = 0.0   # How much the turn rate bounds on ownship and intruder are relaxed
pd=0          # Pilot delay
maxTime = 80  # Initial time (tau) value of reachability (tau counts down)
minTime = -200 # Minimum time value (
               #     Reachability will stop early for convergence or reaching an NMAC)
               #     When time is negative, tau is 0
sets=Dict();   # Dictionary for storing our reachable sets

In [ ]:
# Read the memory-mapped files, which should have already been generated, so this will be fast
# Memory mapping is used because the arrays can be very large
# This works best if using a fast solid state drive
netActions = readNetworkActionsMmap(folder="NetworkApprox",ver=6,token="samp"); #token="reluVal"
reachDynamics = readReachDynamicsMmap(folder="ReachDynamics",ver=1,delta=delta,v0=200.0,v1=180.0);

In [ ]:
# Compute the initial reachable set
# There are other ways to initialize the reachable set
# This method just starts with a single cell
# getInitialSet() adds all cells to the set
sets[maxTime] = getInitialSet_point(pd,x=15000.0,y=0,psi=-pi)

# Define cells that can always be reached at the next time step
# For example, an intruder can always appear at the border of the sensing region
alwaysTrueRegions = [((k==1) || (j==1) || (k==NUMX) || (j==NUMY)) for i=1:NUMP, j=1:NUMY, k=1:NUMX][:];

# We can ignore these regions if we want
alwaysTrueRegions = nothing

# Define the cells that are unsafe (NMAC)
# This might take a few seconds
nmacCells = getNmacCells();

In [ ]:
# Compute reachable sets iteratively. 
# Stop if convergence, an NMAC, or minTime is reached.
for t = 0-1:-1:minTime
    # Compute next reachable set
    @time sets[t] = getNextSetMmap(sets[t+1],netActions,reachDynamics,t,alwaysTrueRegions=alwaysTrueRegions)
    if mod(t,10)==0
        @printf("\nt = %d\n",t)
    end
    
    # Unsafe NMAC defined as reaching an NMAC once tau reaches 0 (t<=0)
    if (t<=0) && (isNmac(sets,t,nmacCells))
        print("We reached an NMAC")
        break
    end
    
    # Check if the previous reachable set is the same as the current reachable set
    # If so, reachability has converged to a steady state solution
    if isConverged(sets,t,verbose=true)
        print("Reached state and no NMACS!")
        break
    end
end

## Plot Results

In [ ]:
# Plot the reachable set
xmin = -10; xmax = 40; ymin = -25; ymax = 25;
t = 0
plotReachable(sets,[xmin,xmax,ymin,ymax],t)

In [ ]:
# Plot an animation of the reachable set changing over time
xmin = -10; xmax = 40; ymin = -25; ymax = 25;
initialTime = 60
finalTime = 50
gifName = "TEMP"
animateReach(sets,[xmin,xmax,ymin,ymax],initialTime,finalTime,gifName)